In [1]:
import pandas as pd
!pip install seaborn
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
output=pd.read_csv('data_with_internal_info.csv', low_memory=False)
rows, columns = output.shape
print("Number of rows:", rows)
print("Number of columns:", columns)

Number of rows: 235277
Number of columns: 60


In [3]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import string
import re
import nltk
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    mean_absolute_percentage_error)
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download("stopwords")
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
# Prepare stopwords and lemmatizer
_stopwords = set(stopwords.words("english"))
_lemmatizer = WordNetLemmatizer()

def text_prep(text_list):
    cleaned = []
    table = str.maketrans('', '', string.punctuation)
    for doc in text_list:
        doc = str(doc).lower().translate(table)
        doc = doc.replace('\n', ' ')
        doc = re.sub(r'[^a-z0-9._\s]', '', doc)
        doc = re.sub(r'\s+', ' ', doc).strip()
        tokens = [tok for tok in doc.split() if tok not in _stopwords and len(tok) > 2]
        lemmas = [_lemmatizer.lemmatize(tok) for tok in tokens]
        cleaned.append(" ".join(lemmas))
    return cleaned
columns_to_combine = [
    'Job Title (en)', 'Job Title (nl)', 'Domain (en)', 'Specialisation (en)',
    'Career ladder en', 'Level Title', 'Function goal',
    'Key result areas: result area (1)', 'Key result areas: result area (2)',
    'Key result areas: result area (3)', 'Key result areas: result area (4)',
    'Key result areas: result area (5)', 'Key result areas: result area (6)',
    'Key result areas: result area (7)', 'Key result areas: result area (8)',
    'Key result areas: result area (9)', 'Key result areas: result area (10)',
    'Key result areas: result area (11)', 'Key result areas: result area (12)',
    'Key result areas: result area (13)', 'Key result areas: result area (14)',
    'Key result areas: result area (15)', 'HIERARCHIC MANAGEMENT',
    'FUNCTIONAL MANAGEMENT', 'Leadership | Manage the following job(s)',
    'Managed by', 'Is he/ she responsible for a certain budget/ figure?',
    'Specify the budget amounts.', 'Diploma', 'Speciality',
    'Required experience', 'Innovation', 'Row 4 - Column 1',
    'Row 5 - Column 1', 'Row 6 - Column 1', 'Row 7 - Column 1',
    'Row 8 - Column 1', 'Row 9 - Column 1', 'Reference Job', 'Age',
    'Diploma Category', 'Job Level', 'Internal Job', 'Internal Job Grade',
    'Department'
]
output[columns_to_combine] = output[columns_to_combine].astype(str)
text_combined = output[columns_to_combine].agg(' '.join, axis=1).tolist()
text_cleaned = text_prep(text_combined)

# Text pipeline: TF-IDF + SVD
def identity_preprocessor(x):
    return x

def whitespace_tokenizer(x):
    return x.split()
text_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=(1,2),
        max_features=2000,
        sublinear_tf=True,
        smooth_idf=True,
        stop_words='english',
        tokenizer=whitespace_tokenizer,
        preprocessor=identity_preprocessor
    )),
    ('svd', TruncatedSVD(n_components=100, random_state=11))
])


# Fit & transform text
X_text = text_pipe.fit_transform(text_cleaned)
X_text = pd.DataFrame(X_text, index=output.index)


X = X_text

y = output['Compas Grade'].fillna(output['Compas Grade'].mean())

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=11,)

# XGBoost regressor
model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.05,
    objective='reg:squarederror',
    random_state=11
)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)


print("--- Evaluation for Compas Grade (TF-IDF + SVD + XGBoost) ---")
print(f"MAE:  {mean_absolute_error(y_test, y_pred):.4f}")
print(f"RMSE: {mean_squared_error(y_test, y_pred) ** 0.5:.4f}")
print(f"R²:   {r2_score(y_test, y_pred):.4f}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, y_pred):.4f}\n")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nazan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nazan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\nazan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


--- Evaluation for Compas Grade (TF-IDF + SVD + XGBoost) ---
MAE:  0.3291
RMSE: 0.4770
R²:   0.9729
MAPE: 0.0304



In [5]:
#Gridsearch on xgboost

model_xgb = xgb.XGBRegressor(
    objective='reg:squarederror',
    tree_method='hist',     
    verbosity=1,
    random_state=11)
param_grid = {
    'n_estimators': [100, 300],
    'max_depth': [3, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 1.0]
}
grid_search = GridSearchCV(
    model_xgb,
    param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=1,
    verbose=2)
grid_search.fit(X_train, y_train)
print("Best hyperparameters for XGBoost:", grid_search.best_params_)
best_xgb = grid_search.best_estimator_

y_true = y_test.values.flatten()
y_pred = best_xgb.predict(X_test)

print("\n--- Evaluation for Compas Grade with XGBoost (tuned) ---")
print("  MAE: ", mean_absolute_error(y_true, y_pred))
print("  RMSE:", mean_squared_error(y_true, y_pred)**0.5)
print("  R²:  ", r2_score(y_true, y_pred))
print("  MAPE:", mean_absolute_percentage_error(y_true, y_pred))

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.6; total time=   2.6s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.6; total time=   2.5s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.6; total time=   2.4s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   2.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   2.3s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   2.3s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=300, subsample=0.6; total time=   6.3s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=300, subsample=0.6; total time=   6.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=300, subsample=0.6; total time=   6.3s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=300, subsample=1.0; total time=   6.2s

In [6]:

#base model of random forest

model_rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=5,
    random_state=11,
    n_jobs=-1
)
X_train.columns = X_train.columns.astype(str)
X_test .columns = X_test .columns.astype(str)
model_rf.fit(X_train, y_train)

y_pred_rf = model_rf.predict(X_test)


# Evaluation
print("\n--- Evaluation for Grade (Random Forest regressors with TF-IDF) ---")
print("  MAE:", mean_absolute_error(y_test, y_pred_rf))
print("  RMSE:", mean_squared_error(y_test, y_pred_rf) ** 0.5)
print("  R²:", r2_score(y_test, y_pred_rf))
print("  MAPE:", mean_absolute_percentage_error(y_test, y_pred_rf))


--- Evaluation for Grade (Random Forest regressors with TF-IDF) ---
  MAE: 0.7067727653933432
  RMSE: 1.0187606054844331
  R²: 0.8763815855091328
  MAPE: 0.07113055287210276


In [7]:
#-------GridSearch for random forest 

param_grid_rf = {
    'n_estimators': [50, 100],
    'max_depth':    [ 5, 10],
    'min_samples_split': [2, 5],
    'max_features': ['log2', 'sqrt']
}

# 4) Set up GridSearchCV
grid_search_rf = GridSearchCV(
    estimator=model_rf,
    param_grid=param_grid_rf,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=1,
    verbose=2
)

# 5) Fit the grid search
grid_search_rf.fit(X_train, y_train)

# 6) Grab best estimator and print best params
best_rf = grid_search_rf.best_estimator_
print("Best hyperparameters for RandomForest:", grid_search_rf.best_params_)

# 7) Evaluate on test set
y_pred_rf = best_rf.predict(X_test)

print("\n--- Evaluation for Grade (Random Forest(Tuned) with TF-IDF) ---")
print("  MAE: ", mean_absolute_error(y_test, y_pred_rf))
print("  RMSE:", mean_squared_error(y_test, y_pred_rf) ** 0.5)
print("  R²:  ", r2_score(y_test, y_pred_rf))
print("  MAPE:", mean_absolute_percentage_error(y_test, y_pred_rf))

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END max_depth=5, max_features=log2, min_samples_split=2, n_estimators=50; total time=   2.9s
[CV] END max_depth=5, max_features=log2, min_samples_split=2, n_estimators=50; total time=   3.7s
[CV] END max_depth=5, max_features=log2, min_samples_split=2, n_estimators=50; total time=   3.2s
[CV] END max_depth=5, max_features=log2, min_samples_split=2, n_estimators=100; total time=   6.6s
[CV] END max_depth=5, max_features=log2, min_samples_split=2, n_estimators=100; total time=   6.5s
[CV] END max_depth=5, max_features=log2, min_samples_split=2, n_estimators=100; total time=   6.4s
[CV] END max_depth=5, max_features=log2, min_samples_split=5, n_estimators=50; total time=   3.2s
[CV] END max_depth=5, max_features=log2, min_samples_split=5, n_estimators=50; total time=   3.3s
[CV] END max_depth=5, max_features=log2, min_samples_split=5, n_estimators=50; total time=   3.4s
[CV] END max_depth=5, max_features=log2, min_samples_s

In [8]:
import joblib
save_dir = "models/Grade"
os.makedirs(save_dir, exist_ok=True)

# 1. Save XGBoost model
best_xgb.save_model(f"{save_dir}/salary_xgboost.json")

# 2. Save text pipeline (TF-IDF + SVD)
joblib.dump(text_pipe, f"{save_dir}/text_pipeline.pkl")

# 3. Save metadata
joblib.dump(columns_to_combine, f"{save_dir}/columns_to_combine.pkl")

print("Grade model saved successfully!")


Grade model saved successfully!
